In [ ]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 로보플로우 설치
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="5vL6l00fJH9ouMEEj0Pu")
project = rf.workspace("project-cyfsl").project("car_plate-ftsum")
dataset = project.version(6).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to car_plate-6 in yolov5pytorch:: 100%|██████████| 1188/1188 [00:00<00:00, 6508.80it/s]


In [ ]:
# 테스트 파일과 학습 파일 리스트 전체 Path를 저장
import glob

DATA_PATH =  "/content/car_plate-4"
TRAIN_PATH = "/content/car_plate-4/train"
VALIDATION_PATH =  "/content/car_plate-4/valid"

train_image_list = glob.glob(TRAIN_PATH + "/images/*.jpg")
validation_image_list = glob.glob(VALIDATION_PATH + "/images/*.jpg")

In [ ]:
# train.txt(validation.txt) 파일을 생성 한 후에 앞서 저장된 train_image_list(validation_image_list) 파일 리스트를 저장
with open(DATA_PATH + '/train.txt', 'w') as f:
    f.write('\n'.join(train_image_list) + '\n')

with open(DATA_PATH + '/validation.txt', 'w') as f:
    f.write('\n'.join(validation_image_list) + '\n')

In [ ]:
# yaml 데이터를 열어서(로딩) data라는 변수에 정보를 딕셔너리 형태로 저장
import yaml

with open('/content/car_plate-4/data.yaml', 'r') as f:
  data = yaml.load(f, Loader=yaml.FullLoader)

print(data)
print(type(data))

{'names': ['car', 'plate'], 'nc': 2, 'roboflow': {'license': 'CC BY 4.0', 'project': 'car_plate-ftsum', 'url': 'https://universe.roboflow.com/project-cyfsl/car_plate-ftsum/dataset/4', 'version': 4, 'workspace': 'project-cyfsl'}, 'test': '../test/images', 'train': '/content/car_plate-4/train.txt', 'val': '/content/car_plate-4/validation.txt'}
<class 'dict'>


In [ ]:
# # train Key값에는 train.txt 파일을 val key 값에는 validation.txt를 넣음
data['train'] = DATA_PATH + '/train.txt'
data['val'] = DATA_PATH + '/validation.txt'

# 변경된 'data' dictionary 를 Yaml 파일로 변경하여 저장
with open(DATA_PATH + '/data.yaml', 'w') as f:
  yaml.dump(data, f)

In [ ]:
# 깃허브에서 yolov5 모델 복사
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
# yolov5 폴더로 현 폴더를 이동
%cd yolov5

/content/yolov5/yolov5/yolov5/yolov5


In [ ]:
# yolov5안의 requirements.txt 내부 패키지 설치
!pip install -r requirements.txt

In [ ]:
# 모델 학습
!python train.py --img 640 --batch 16 --epochs 100 --data /content/car_plate-4/data.yaml --weights yolov5x6.pt --name platev1 --exist-ok

2024-01-04 04:19:24.400022: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 04:19:24.415153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 04:19:24.417158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x6.pt, cfg=, data=/content/car_plate-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, 

- 테스트

In [ ]:
# test image 경로 설정
TEST_IMAGE_PATH = "/content/drive/MyDrive/빅프_자료/image"

- 이미지 디텍팅

In [ ]:
# 학습한 모델로 test image 적용
!python detect.py --source '{TEST_IMAGE_PATH}' --weights /content/yolov5/runs/train/platev1/weights/best.pt --img 416 --conf 0.5 --project /content/ --name=result_imagev1 --exist-ok --line-thickness 2

detect: weights=['/content/yolov5/runs/train/platev1/weights/best.pt'], source=/content/drive/MyDrive/빅프_자료/image, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/, name=result_imagev1, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-254-gba63208 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86186872 parameters, 0 gradients, 203.8 GFLOPs
Traceback (most recent call last):
  File "/content/yolov5/detect.py", line 285, in <module>
    main(opt)
  File "/content/yolov5/detect.py", line 280, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_

In [ ]:
# 이미지 리스트 가져오기
img_list = glob.glob("/content/result_imagev1/*.jpg")

In [ ]:
# 적용시킨 이미지 화면에 출력
from google.colab.patches import cv2_imshow
import os
import glob
import cv2
import matplotlib.pyplot as plt
%matplotlib inline


import matplotlib.pyplot as plt

# figure의 사이즈를 정의하고 몇 바이 몇 행렬에 그림들을 뿌릴 것인지 결정 (본 자료는 20개 이므로 4*5임)
fig = plt.figure(figsize=(10, 20))
rows = 2
cols = 2
i= 1

for filename in img_list:
  #이미지 파일들을 하나씩 읽어서 img2에 넣고
  img2 = cv2.imread(filename)

  #4X5 매트릭스 플롯에 하나씩 넣어 보이기
  ax = fig.add_subplot(rows, cols, i)
  ax.imshow(img2)

  # 눈금 제거
  ax.set_xticks([]), ax.set_yticks([])
  i +=1

plt.show()

<Figure size 1000x2000 with 0 Axes>

- 영상 디텍팅

In [54]:
# test video 경로
TEST_VIDEO_PATH = "/content/drive/MyDrive/빅프_자료/video/재원오빠혼자탈출.mp4"

In [55]:
# 학습한 모델로 test video 적용
!python detect.py --source '{TEST_VIDEO_PATH}' --weights /content/yolov5/runs/train/platev1/weights/best.pt --img 416 --conf 0.8 --project /content/ --name=result_videov1 --exist-ok --line-thickness 2

detect: weights=['/content/yolov5/runs/train/platev1/weights/best.pt'], source=/content/drive/MyDrive/빅프_자료/video/재원오빠혼자탈출.mp4, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.8, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/, name=result_videov1, exist_ok=True, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-267-gc42aba5 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 416 layers, 139980484 parameters, 0 gradients, 207.9 GFLOPs
WARNING ⚠️ --img-size [416, 416] must be multiple of max stride 64, updating to [448, 448]
video 1/1 (1/1096) /content/drive/MyDrive/빅프_자료/video/재원오빠혼자탈출.mp4: 256x448 2 cars, 76.5ms
video 1/1 (2/1096) /content/drive/MyDrive/빅프_자료/vid

In [ ]:
# # 압축하기
# detect_video = '/content/drive/MyDrive/bigproject/video/car1.mp4'

# def video_compressing(detect_video):
#   compressed_video = detect_video[:-4] + "_compressed.mp4"
#   os.system(f"ffmpeg -i {detect_video} -vcodec libx264 {compressed_video}")

#   return compressed_video

In [ ]:
# # 압축한 파일 화면에 출력하기
# compressed_video = video_compressing(detect_video)

# from IPython.display import HTML
# from base64 import b64encode

# mp4 = open(compressed_video,'rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML("""
# <video width=400 controls>
#     <source src="%s" type="video/mp4">
# </video>
# """ % data_url)

In [ ]:
# 모델 저장
!cp /content/yolov5/runs/train/platev1/weights/best.pt /content/drive/MyDrive/빅프_자료